# ToDo:
- проверить поиск с пам

In [1]:
from Bio import Entrez
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
Entrez.email = 'akgribkova@gmail.com' 
import re

In [2]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO
from Bio.Blast import NCBIXML

## примеры qblast

In [ ]:
#help(NCBIWWW.qblast)

The degenerate nucleotide codes in red are treated as mismatches in nucleotide alignment. Too many such degenerate codes within an input nucleotide query will cause the BLAST webpage to reject the input. For protein queries, too many nucleotide-like code (A,C,G,T,N) may also cause similar rejection.
K  G/T (keto)         S  G/C (strong)         Y  T/C (pyrimidine) 
		M  A/C (amino)        W  A/T (weak)           R  G/A (purine)        
		B  G/T/C              D  G/A/T                H  A/C/T      
		V  G/C/A              -  gap of indeterminate length

- For nucleotide-nucleotide searches (i.e., "blastn") an exact match of the entire word is required before an extension is initiated, so that one normally regulates the sensitivity and speed of the search by increasing or decreasing the word-size. 
- Reward and Penalty for Nucleotide Programs
Many nucleotide searches use a simple scoring system that consists of a "reward" for a match and a "penalty" for a mismatch. The (absolute) reward/penalty ratio should be increased as one looks at more divergent sequences. **A ratio of 0.33 (1/-3) is appropriate for sequences that are about 99% conserved DEFAULT**; a ratio of 0.5 (1/-2) is best for sequences that are 95% conserved; a ratio of about one (1/-1) is best for sequences that are 75% conserved [1]. Read more here [1] States DJ, Gish W, and Altschul SF (1991) METHODS: A companion to Methods in Enzymology 3:66-70.

 NCBIWWW.qblast("blastn", "nt", 'TTACAGTTAAGCTACCTTATGC', nucl_reward=1, nucl_penalty=-2, hitlist_size=1000, word_size = 7) 
 - получилось 137 таргетов с hsp.gaps ==0

The "Search for short, nearly exact matches" nucleotide and protein pages no longer exist. Instead, the nucleotide and protein blast programs automatically check for short queries and adjust the search parameters accordingly. This adjustment occurs when the query, either nucleotide or amino acid, is of length 30 or less. The translating blast programs or searches on the genome blast pages do not have this auto adjust feature.

In [3]:
%%time
result_handle = NCBIWWW.qblast("blastn", "nt", 'TAATTTCTACTAAGTGTAGATCCCCCAGCGCTTCAGCGTTC', nucl_reward=1, nucl_penalty=-1, hitlist_size=1000, word_size = 7, expect=200000.0 )
                              #filter=False,
                              #composition_based_statistics=False,
                              #matrix_name="PAM30",) #NNGRRT #word_size = 11,
blast_record = NCBIXML.read(result_handle)


KeyboardInterrupt: 

In [6]:
%%time
result_handle = NCBIWWW.qblast("blastn", "nt", 'CACGTTGAGACAGCGTCCAA', nucl_reward=1, nucl_penalty=-3, hitlist_size=1000, word_size = 7, expect=1000 )
                              #filter=False,
                              #composition_based_statistics=False,
                              #matrix_name="PAM30",) #NNGRRT #word_size = 11,
blast_record = NCBIXML.read(result_handle)

CPU times: user 223 ms, sys: 30.1 ms, total: 253 ms
Wall time: 1min 3s


In [ ]:
#hitlist_size   Number of hits to return. Default 50
#ncbi_gi= True, 

In [188]:
#with open("targe_blast.xml", "w") as out_handle:
#    out_handle.write(result_handle.read())
#result_handle.close()
#result_handle = open("targe_blast.xml")

- если не хотим смотреть глазами на организмы, в которых найдены таргеты - прописываем gi тех, кого не хотим видеть

In [38]:
# не наш случай for multiple query sequences
#blast_records = NCBIXML.parse(result_handle)

In [8]:
E_VALUE_THRESH = 1000


In [9]:
#a = []
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:  
            print("****Alignment****")
            print("sequence:", alignment.title)
            print("length:", alignment.length)
            print("e value:", hsp.expect)
            print(hsp.query )
            print(hsp.match )
            print(hsp.sbjct )
            print(hsp.gaps)
            #print(hsp.score )
            #print(hsp.query_start )
            #print(hsp.sbjct_start )
            #print(hsp.strand )
            #a.append(hsp.gaps)
            

****Alignment****
sequence: gi|1782081900|gb|CP046389.1| Borrelia miyamotoi strain CZ-F1E chromosome, complete genome
length: 904129
e value: 0.52973
CACGTTGAGACAGCGTCCAA
||||||||||||||||||||
CACGTTGAGACAGCGTCCAA
0
****Alignment****
sequence: gi|1782081117|gb|CP046388.1| Borrelia miyamotoi strain CZ-F190E chromosome, complete genome
length: 904095
e value: 0.52973
CACGTTGAGACAGCGTCCAA
||||||||||||||||||||
CACGTTGAGACAGCGTCCAA
0
****Alignment****
sequence: gi|1761780990|gb|CP044783.1| Borrelia miyamotoi strain NL-IR-1 chromosome, complete genome
length: 903844
e value: 0.52973
CACGTTGAGACAGCGTCCAA
||||||||||||||||||||
CACGTTGAGACAGCGTCCAA
0
****Alignment****
sequence: gi|1761779645|gb|CP044625.1| Borrelia miyamotoi strain NL-IR-2 chromosome, complete genome
length: 905334
e value: 0.52973
CACGTTGAGACAGCGTCCAA
||||||||||||||||||||
CACGTTGAGACAGCGTCCAA
0
****Alignment****
sequence: gi|1758968360|gb|CP044535.1| Borrelia sp. CA690 chromosome, complete genome
length: 902176
e value: 0.52973


In [17]:
len(a)

NameError: name 'a' is not defined

## Подгружаем готовые таргеты из другого ноутбука

In [11]:
def id_search(seq_id):
    handle=Entrez.efetch(db="nucleotide", id = seq_id, rettype='fasta')  
    for seq_record in SeqIO.parse(handle,'fasta'):
        return seq_record.seq

In [12]:
cas_info = {'SpCas9': {'pam_forw' : '[ACGT]GG', 'pam_rev' : 'CC[ACGT]', 'pam_len':3, 'len_target' : 20},
            'SaCas9': {'pam_forw' : '[ACGT][ACGT]G[AG][AG]T', 'pam_rev' : 'A[TC][TC]C[ACGT][ACGT]', 'pam_len':6, 'len_target' : 21}, 
            'CjCas9': {'pam_forw' : '[ACGT][ACGT][ACGT][ACGT]A[CT]AC', 'pam_rev' : 'GT[GA]T[ACGT][ACGT][ACGT][ACGT]', 'pam_len':8, 'len_target' : 22}, 
            'StCas9': { 'pam_forw' : '[ACGT][ACGT]AGAA[AT]', 'pam_rev' : '[CG]TTCT[ACGT][ACGT]', 'pam_len':7, 'len_target' : 20} }

In [13]:
cas_1 = 'CjCas9'
cas_2 = 'SaCas9'
orientation = 'PAM-in'

In [14]:
off_target_genome = id_search(alignment.title.split('|')[3])

if hsp.strand[1] == 'Minus':
    possible_pam = off_target_1[hsp.sbjct_start - cas_info[cas_1]['pam_len']:hsp.sbjct_start] 
else:
    possible_pam = off_target_1[hsp.sbjct_start + len(hsp.sbjct) : hsp.sbjct_start + len(hsp.sbjct) + cas_info[cas_1]['pam_len']:] 
res = re.match(cas_info[cas_1]['pam_rev'], str(possible_pam))         #str(possible_pam))
if res:
    print('yes')
else:
    print('no')

NameError: name 'off_target_1' is not defined

In [15]:
final = {}
final['CjCas9_SaCas9_PAM-in'] = {1: {'left': {'afzelii': {35743: 'GCTTTTTAACTATTTGATCTCT'},
   'bavariensis': {35777: 'GCTTTTTAACTATTTGATCTCT'},
   'burgdorferi': {35898: 'GCTTTTTAACTATTTGATCTCT'},
   'garenii': {35908: 'GCTTTTTAACTATTTGATCTCT'}},
  'right': {'afzelii': {35743: 'TTAATTTTAAAACATCTATCAA'},
   'bavariensis': {35777: 'TTAATTTTAAAACATCTATCAA'},
   'burgdorferi': {35898: 'TTAATTTTAAAACATCTATCAA'},
   'garenii': {35908: 'TTAATTTTAAAATATCTATCAA'}}},
 2: {'left': {'afzelii': {436838: 'TTACAGTTAAGCTACCTTATGC'},
   'bavariensis': {436536: 'TTACAGTTAAGCTACCTTATGC'},
   'burgdorferi': {439384: 'TTACAGTTAAGCTACCTTATGC'},
   'garenii': {435891: 'TTACAGTTAAGCTACCTTATGC'}},
  'right': {'afzelii': {436838: 'AGGTAACCTTTGCGCACCTCCG'},
   'bavariensis': {436536: 'AGGTAACCTTTGCGCACCTCCG'},
   'burgdorferi': {439384: 'AGGTAACCTTTGCGCACCTCCG'},
   'garenii': {435891: 'AGGTAACCTTTGCGCACCTCCG'}}},
 3: {'left': {'afzelii': {436878: 'CACGTTGAGACAGCGTCCAAAT'},
   'bavariensis': {436233: 'CACGTTGAGACAGCGTCCAAAT'},
   'burgdorferi': {436490: 'CACGTTGAGACAGCGTCCAAAT'},
   'garenii': {437180: 'CACGTTGAGACAGCGTCCAAAT'}},
  'right': {'afzelii': {436878: 'CAAGGAATTTCGCTACCTTAGG'},
   'bavariensis': {436233: 'CAAGGAATTTCGCTACCTTAGG'},
   'burgdorferi': {436490: 'CAAGGAATTTCGCTACCTTAGG'},
   'garenii': {437180: 'CAAGGAATTTCGCTACCTTAGG'}}},
 4: {'left': {'afzelii': {436838: 'TTACAGTTAAGCTACCTTATGC'},
   'bavariensis': {436536: 'TTACAGTTAAGCTACCTTATGC'},
   'burgdorferi': {439384: 'TTACAGTTAAGCTACCTTATGC'},
   'garenii': {435891: 'TTACAGTTAAGCTACCTTATGC'}},
  'right': {'afzelii': {436838: 'AGGTAACCTTTGCGCACCTCCG'},
   'bavariensis': {436536: 'AGGTAACCTTTGCGCACCTCCG'},
   'burgdorferi': {439384: 'AGGTAACCTTTGCGCACCTCCG'},
   'garenii': {435891: 'AGGTAACCTTTGCGCACCTCCG'}}}, }

In [16]:
[*final['CjCas9_SaCas9_PAM-in'][2]['left']['afzelii'].values() ][0][:12]

'TTACAGTTAAGC'

In [17]:
def pathogen_targets_alignment(seq1, seq2, mismatches_thres_12, mismatches_thres_all ):

    mismatches_thres_12 = 2
    mismatches_thres_all = 4

    alignments = aligner.align(seq1[:12], seq2[:12])
    for alignment in alignments:
        if 12- alignment.score <= mismatches_thres_12:
            alignments = aligner.align(seq1, seq2)
            for alignment in alignments:
                if min(len(seq1), len(seq2)) - alignment.score <= mismatches_thres_all:
                    return True

In [14]:
pat_target_left = [*final['CjCas9_SaCas9_PAM-in'][3]['left']['afzelii'].values() ][0][:12]
result_handle = NCBIWWW.qblast("blastn", "nt", pat_target_left) #NNGRRT
blast_record = NCBIXML.read(result_handle)

NameError: name 'NCBIWWW' is not defined

In [153]:
pat_target_left

'CACGTTGAGACA'

In [149]:
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print(hsp.expect)

In [137]:

def blast_search_targets(final ):

    #for num in final['CjCas9_SaCas9_PAM-in'].keys():
    for num in [4]:
        print('start {}'.format(num))
        # взяли первые 12-ть нуклеотидов первого таргет 
        pat_target_left = [*final['CjCas9_SaCas9_PAM-in'][num]['left']['afzelii'].values() ][0][:12]
        print('start blast')
        
        result_handle = NCBIWWW.qblast("blastn", "nt", pat_target_left, nucl_reward=1, nucl_penalty=-2, hitlist_size=100, word_size = 7)
        #result_handle = NCBIWWW.qblast("blastn", "nt", pat_target_left) #NNGRRT
        blast_record = NCBIXML.read(result_handle)
        print('end blast')
        for alignment in blast_record.alignments:
            possible_species = alignment.title
            # подгрузили геном off species
            off_target_genome = id_search(alignment.title.split('|')[3])
            print('start PAM searching in off_target_genome {}'.format(possible_species))
            for hsp in alignment.hsps:
                if hsp.expect < E_VALUE_THRESH:
                    off_target_start_1 = hsp.sbjct_start
                    # нашли пам для любого из двух кас в зависимости от того, на какой цепи таргет
                    
                    
                    if hsp.strand[1] == 'Minus':
                        possible_pam = off_target_genome[hsp.sbjct_start - cas_info[cas_1]['pam_len']:hsp.sbjct_start] 
                    else:
                        possible_pam = off_target_genome[hsp.sbjct_start + len(hsp.sbjct) : hsp.sbjct_start + len(hsp.sbjct) + cas_info[cas_1]['pam_len']] 
                    
                    res = re.match(cas_info[cas_1]['pam_rev'], str(possible_pam))   
                    if res:
                        print('Result for matching 2 Cas proteins on left pat target: {}'.format(decision) )
                            
                        pat_target_right = [*final['CjCas9_SaCas9_PAM-in'][num]['right']['afzelii'].values() ][0][:12]
                        print('start blast for second target')
                        result_handle = NCBIWWW.qblast("blastn", "nt", pat_target_left) #NNGRRT
                        blast_record = NCBIXML.read(result_handle)
                        print('end blast for second target')
                        for alignment in blast_record.alignments:
                            if alignment.title == possible_species:
                                for hsp in alignment.hsps:
                                    if hsp.expect < E_VALUE_THRESH:
                                        off_target_start_2 = hsp.sbjct_start
                                            
     
                                        if hsp.strand[1] == 'Minus':
                                            possible_pam = off_target_genome[hsp.sbjct_start - cas_info[cas_2]['pam_len']:hsp.sbjct_start] 
                                        else:
                                            possible_pam = off_target_genome[hsp.sbjct_start + len(hsp.sbjct) : hsp.sbjct_start + len(hsp.sbjct) + cas_info[cas_2]['pam_len']:]
                                                
                                        res = re.match(cas_info[cas_1]['pam_rev'], str(possible_pam))   
                                        if res:
                                            print('Result for matching 2 Cas proteins on right pat target: {}'.format(decision) )

                                            if abs(off_target_start_1 -off_target_start_2 ) <100:
                                                print('___________________')
                                                print(possible_species)
                                                print(off_target_start_1, off_target_start_2)
    
    
    
    return()

In [ ]:
%%time

blast_search_targets(final)

start 1
start blast
end blast
start 2
start blast
end blast
start 3
start blast
end blast
start 4
start blast


In [17]:
list_of_cross_species

{'Borrelia afzelii',
 'Borrelia burgdorferi',
 'Borrelia garinii',
 'Borrelia sp.',
 'Borreliella afzelii',
 'Borreliella bavariensis',
 'Borreliella burgdorferi',
 'Borreliella garinii',
 'Borreliella valaisiana',
 'Lachnoanaerobaculum umeaense',
 'Paraglaciecola sp.',
 'Pseudobutyrivibrio xylanivorans',
 'Sporosarcina pasteurii',
 'Virgibacillus sp.'}

# STANDALONE BLAST

In [10]:
from Bio import SeqIO
from Bio.Blast.Applications import NcbiblastpCommandline,NcbiblastnCommandline
from Bio.Blast import NCBIXML
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import uuid,os,re


In [18]:
%%time
list_of_fasta = []
for counter, value in enumerate(final['CjCas9_SaCas9_PAM-in'].keys()):
    pat_target_left = [*final['CjCas9_SaCas9_PAM-in'][value]['left']['afzelii'].values() ][0]
    list_of_fasta.append(pat_target_left)

records = (SeqRecord(Seq(seq), str(index)) for index,seq in enumerate(list_of_fasta) )
SeqIO.write(records, 'targets_list.fasta', "fasta")

       

CPU times: user 2.47 ms, sys: 0 ns, total: 2.47 ms
Wall time: 67.9 ms


In [33]:
%%time

blastn_cline = NcbiblastnCommandline(query='targets_list.fasta',
                                            db='nt',
                                            outfmt=5,
                                            out='blast_result.xml',
                                            num_threads=20,
                                            word_size=11,
                                     evalue=1000, strand='both',
                                     template_type = 'coding', template_length = 20, reward=1, penalty=-3)
stdout, stderr = blastn_cline(env={'BLASTDB':'/home/_shared/nt_raw/'})

CPU times: user 7.88 ms, sys: 8.54 ms, total: 16.4 ms
Wall time: 1min 34s


In [41]:
with open("blast_result.xml") as b:
    for record in NCBIXML.parse(b):
            record.alignments = record.alignments
            print('_______________________________________________________________________')
            print('_______________________________________________________________________')
            print('_______________________________________________________________________')
            for alignment in record.alignments:
                
                for hsp in alignment.hsps:
                    if hsp.expect < 1000:
                     #if hsp.gaps >0:
                        print("****Alignment****")
                        print("sequence:", alignment.title)
                        print("length:", alignment.length)
                        print("e value:", hsp.expect)
                        print(hsp.query )
                        print(hsp.match )
                        print(hsp.sbjct )
                        print(hsp.gaps)
                        
                        print(hsp.positives)
            #print(hsp.score )
            #print(hsp.query_start )
            #print(hsp.sbjct_start )
            #print(hsp.strand )
            #a.append(hsp.gaps)

_______________________________________________________________________
_______________________________________________________________________
_______________________________________________________________________
****Alignment****
sequence: gb|CP044535.1| Borrelia sp. CA690 chromosome, complete genome
length: 902176
e value: 0.0325785
GCTTTTTAACTATTTGATCTCT
||||||||||||||||||||||
GCTTTTTAACTATTTGATCTCT
0
22
****Alignment****
sequence: gb|CP028861.1| Borreliella garinii strain 20047 chromosome, complete genome
length: 906449
e value: 0.0325785
GCTTTTTAACTATTTGATCTCT
||||||||||||||||||||||
GCTTTTTAACTATTTGATCTCT
0
22
****Alignment****
sequence: gb|CP028872.1| Borreliella bavariensis PBi chromosome 1, complete sequence
length: 905817
e value: 0.0325785
GCTTTTTAACTATTTGATCTCT
||||||||||||||||||||||
GCTTTTTAACTATTTGATCTCT
0
22
****Alignment****
sequence: gb|CP031412.1| Borreliella burgdorferi strain MM1 chromosome main, complete sequence
length: 908512
e value: 0.0325785
GCTTTTTAACTATTTG

In [ ]:
E_VALUE_THRESH = 1e-20 

for record in NCBIXML.parse(open("results.xml")): 
    if record.alignments: 
        print("\n") 
        print("query: %s" % record.query[:100]) 
        for align in record.alignments: 
            for hsp in align.hsps: 
                if hsp.expect < E_VALUE_THRESH: 
                    print("match: %s " % align.title[:100])

In [91]:
%%time

E_VALUE_THRESH = 0.04

cross_dict = {}
for target in targets_left:
    list_of_cross_species = []
    
    result_handle = NCBIWWW.qblast("blastn", "nt", target)
    
    with open("targe_blast.xml", "w") as out_handle:
        out_handle.write(result_handle.read())
    result_handle.close()
    
    result_handle = open("targe_blast.xml")
    
    blast_record = NCBIXML.read(result_handle)
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            if hsp.expect < E_VALUE_THRESH:
                list_of_cross_species = list_of_cross_species.add(alignment.title) #.split('|')[4]
    cross_dict[target] = list_of_cross_species

KeyboardInterrupt: 

Candidates whose PAM-proximal
12 bp sequence appeared in the HOMG collection were subsequently excluded. An algorithm of CRISPR Design tool
(ref 20 and http://crispr.mit.edu/about) was used to score the
off-target effect for the remaining target candidates. The
candidates were then ranked with an index scored from 100%
to 0% (Supplemental Figure S4), and the ones with a score
lower than 45% were further removed.

In [111]:
blast_record

In [112]:
blast_record = NCBIXML.read(open('blast_result.xml','r'))
for alignment in blast_record.alignments:
    print(alignment)

FileNotFoundError: [Errno 2] No such file or directory: 'blast_result.xml'

# Gene mapping

In [57]:
from Bio.SeqUtils.Mapper import MapPosition

def use_genbank(self):
    return str(self.to_genbank())
MapPosition.__str__ = use_genbank

ImportError: No module named 'Bio.SeqUtils.Mapper'